<a href="https://colab.research.google.com/github/SonalDhoke/Programming_with_Data_Analysis_PY/blob/main/st20336540_CMP7005_PDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing the necessary libraries for data analysis
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
drive_path ='/content/drive/MyDrive/Colab/Programming_DA/PDA_Assessment/Assessment Data-20251118'

In [5]:
dataframes = []
for filename in os.listdir(drive_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV file
        file_path = os.path.join(drive_path, filename)
        df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame
        dataframes.append(df)  # Add the DataFrame to the list

In [6]:
df = pd.concat(dataframes, ignore_index=True)
df

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Brajrajnagar,07/12/2017,171.38,311.68,NaN,41.07,NaN,7.78,3.26,5.83,4.69,0.00,NaN,NaN,NaN,NaN
1,Brajrajnagar,08/12/2017,126.67,192.81,NaN,20.16,NaN,7.97,2.94,5.43,10.51,0.00,NaN,NaN,304.0,Very Poor
2,Brajrajnagar,09/12/2017,105.11,154.39,NaN,18.40,NaN,8.15,2.83,5.10,10.33,0.00,NaN,NaN,291.0,Poor
3,Brajrajnagar,10/12/2017,65.25,99.73,NaN,15.21,NaN,7.90,2.74,4.75,8.37,0.00,NaN,NaN,189.0,Moderate
4,Brajrajnagar,11/12/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29526,Ahmedabad,27/06/2020,62.12,118.67,9.18,56.35,19.86,NaN,0.49,12.44,68.05,1.32,37.76,1.62,92.0,Satisfactory
29527,Ahmedabad,28/06/2020,31.57,NaN,6.37,23.99,16.40,NaN,0.52,11.01,26.34,1.37,49.58,1.34,82.0,Satisfactory
29528,Ahmedabad,29/06/2020,29.75,127.98,9.06,25.15,18.92,NaN,0.67,12.10,34.99,1.39,60.21,0.79,74.0,Satisfactory
29529,Ahmedabad,30/06/2020,40.02,121.10,7.09,58.92,33.41,NaN,0.73,16.39,41.64,1.21,44.10,1.35,98.0,Satisfactory


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [8]:
# Make a copy of original data BEFORE any cleaning
df_raw = df.copy()

In [9]:
print(f"There are  {df.duplicated().sum()} duplicates in the dataset")

There are  0 duplicates in the dataset


In [10]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [11]:
#checking Missing values
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * mis_val / len(df)

    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    mis_val_table = mis_val_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table = mis_val_table.sort_values('% of Total Values', ascending=False)

    return mis_val_table

missing_values = missing_values_table(df)
display(missing_values.style.background_gradient(cmap='Greens'))


,Missing Values,% of Total Values
Xylene,18109,61.322001
PM10,11140,37.723071
NH3,10328,34.973418
Toluene,8041,27.229014
Benzene,5623,19.041008
AQI,4681,15.851139
AQI_Bucket,4681,15.851139
PM2.5,4598,15.570079
NOx,4185,14.171549
O3,4022,13.619586


In [12]:
# 1) Drop Xylene  because more than 50 % data is missing
#Why drop Xylene now?
#When >50% of values are missing, any imputation will mainly be guesswork and will bias downstream results (source apportionment, clustering, AQI drivers)
df = df.drop(columns=["Xylene"])

In [13]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values(["City", "Date"])

cols_to_interp = ["PM2.5", "PM10", "CO"]

def interpolate_city(group):
    group = group.set_index("Date") #temporary index creation
    for col in cols_to_interp:
        group[col] = group[col].interpolate(
            method="time",
            limit=3,          # fill small gaps
            limit_direction="both"
        )
    return group.reset_index()

df = df.groupby("City", group_keys=False).apply(interpolate_city)
print(df[cols_to_interp].isna().sum())


PM2.5     3869
PM10     10368
CO        1238
dtype: int64


/tmp/ipython-input-257547496.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("City", group_keys=False).apply(interpolate_city)


In [14]:
# Sort by Date only
df = df.sort_values("Date")

def interpolate_date(df):
    df2 = df.set_index("Date")
    for col in cols_to_interp:
        df2[col] = df2[col].interpolate(
            method="time",
            limit_direction="both"
        )
    return df2.reset_index()

df = interpolate_date(df)
print(df[cols_to_interp].isna().sum())


PM2.5    0
PM10     0
CO       0
dtype: int64
